In [1]:
import pandas as pd
import numpy as np
from random import shuffle
from osgeo import ogr, osr
from sentinelhub import WmsRequest, WcsRequest, MimeType, CRS, BBox, constants, DataSource, CustomUrlParam
from s2cloudless import S2PixelCloudDetector, CloudMaskRequest
import logging
from collections import Counter
import datetime
import os
import yaml

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.transform import resize

import scipy.sparse as sparse
from scipy.sparse.linalg import splu


with open("../../config.yaml", 'r') as stream:
        key = (yaml.safe_load(stream))
        API_KEY = key['key'] 
        
%run ../../src/slope.py
%run ../../src/utils-bilinear.py
%run ../../src/dsen2/utils/DSen2Net.py

/Users/john.brandt/anaconda3/envs/remote_sensing/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.


In [2]:
df = pd.read_csv("../../data/stratified.csv")
df['PLOT_ID'] = range(0, len(df))

In [3]:
time = ('2018-12-15', '2020-01-15')
EPSG = CRS.WGS84
IMSIZE = 48
cloud_detector = S2PixelCloudDetector(threshold=0.4, average_over=4, dilation_size=2)

# Helper functions (to be moved to a utils file)

In [4]:
def calculate_proximal_steps_index(date, satisfactory):
    arg_before = None
    arg_after = None
    if date > 0:
        idx_before = satisfactory - date
        arg_before = idx_before[np.where(idx_before < 0, idx_before, -np.inf).argmax()]
    if date < np.max(satisfactory):
        idx_after = satisfactory - date
        arg_after = idx_after[np.where(idx_after > 0, idx_after, np.inf).argmin()]
    if not arg_after and not arg_before:
        arg_after = date
        arg_before = date
    if not arg_after:
        arg_after = arg_before
    if not arg_before:
        arg_before = arg_after
    return arg_before, arg_after

def speyediff(N, d, format = 'csc'):
    shape = (N-d, N)
    diagonals = np.zeros(2*d + 1)
    diagonals[d] = 1.
    for i in range(d):
        diff = diagonals[:-1] - diagonals[1:]
        diagonals = diff
    offsets = np.arange(d+1)
    spmat = sparse.diags(diagonals, offsets, shape, format = format)
    return spmat

def smooth(y, lmbd, d = 2):
    m = len(y)
    E = sparse.eye(m, format = 'csc')
    D = speyediff(m, d, format = 'csc')
    coefmat = E + lmbd * D.conj().T.dot(D)
    z = splu(coefmat).solve(y)
    return z

MDL_PATH = "../../src/dsen2/models/"

input_shape = ((4, None, None), (6, None, None))
model = s2model(input_shape, num_layers=6, feature_size=128)
predict_file = MDL_PATH+'s2_032_lr_1e-04.hdf5'
print('Symbolic Model Created.')

model.load_weights(predict_file)

def DSen2(d10, d20):
    test = [d10, d20]
    input_shape = ((4, None, None), (6, None, None))
    prediction = _predict(test, input_shape, deep=False)
    #prediction *= 5
    return prediction

def _predict(test, input_shape, model = model, deep=False, run_60=False):
    
    print("Predicting using file: {}".format(predict_file))
    prediction = model.predict(test, verbose=1)
    return prediction

c_arr = np.array([[1, 1, 1, 1, 1,],
                  [1, 2, 2, 2, 1,],
                  [1, 2, 3, 2, 1,],
                  [1, 2, 2, 2, 1,],
                  [1, 1, 1, 1, 1,],])
                  
c_arr = c_arr / 3
o_arr = 1 - c_arr
c_arr = np.tile(c_arr[:, :, np.newaxis], (1, 1, 11))
o_arr = np.tile(o_arr[:, :, np.newaxis], (1, 1, 11))

Symbolic Model Created.


# Bounding boxes

In [5]:
def convertCoords(xy, src='', targ=''):

    srcproj = osr.SpatialReference()
    srcproj.ImportFromEPSG(src)
    targproj = osr.SpatialReference()
    if isinstance(targ, str):
        targproj.ImportFromProj4(targ)
    else:
        targproj.ImportFromEPSG(targ)
    transform = osr.CoordinateTransformation(srcproj, targproj)

    pt = ogr.Geometry(ogr.wkbPoint)
    pt.AddPoint(xy[0], xy[1])
    pt.Transform(transform)
    return([pt.GetX(), pt.GetY()])

def calculate_epsg(coord):
    ''' Returns a UTM EPSG for a lat, long coord'''
    ___xx = 10 + ((coord[0] - (-126)) // 6)
    __x__ = 6 if coord[1] > 0 else 7
    return int( str(32) + str(__x__) + str(___xx)[:-2] )

def calc_bbox(plot_id, df):
    print(plot_id)
    subs = df[df['PLOT_ID'] == plot_id]
    # TOP, LEFT, BOTTOM, RIGHT
    # (min x, min y), (max x, max y)
    return [(min(subs['location_x']), min(subs['location_y'])),
            (max(subs['location_x']), max(subs['location_y']))]

def bounding_box(points, expansion = 160):
    # LONG, LAT FOR SOME REASON
    middle = points  
    epsg = calculate_epsg(middle)

    middle = convertCoords(middle, 4326, epsg)
    expansion1 = 75
    expansion2 = 85
    bl = [middle[0] - 235, middle[1] - 235]
    tr = [middle[0] + 245, middle[1] + 245]

    after = [b - a for a,b in zip(bl, tr)]   
    print(np.array(bl) - np.array(middle))
    print(np.array(tr) - np.array(middle))
    
    bl = convertCoords(bl, epsg, 4326)
    tr = convertCoords(tr, epsg, 4326)
    return bl, tr

# Data download

In [6]:
def rank_array(arr):
    order = arr.argsort()
    ranks = order.argsort()
    return ranks

def mcm_shadow_mask(arr, c_probs):
    #' From "Cloud and cloud shadow masking for Sentinel-2 using multi-
    #         temporal images in global area. Candra et al. 2020."
    #'[B02,B03,B04, B08, B05,B06,B07, B8A,B11,B12]
    #'[B01,B02,B04,B05,B08,B8A,B09,B10,B11,B12]
    #'[ 0 , 1 , 2 , 3  , 4  , 5,  6   7  , 8  , 9]'
    mean_c_probs = np.mean(c_probs, axis = (1, 2))
    cloudy_steps = np.argwhere(mean_c_probs > 0.25)
    images_clean = np.delete(arr, cloudy_steps, 0)
    cloud_ranks = rank_array(mean_c_probs)
    diffs = abs(np.sum(arr - np.mean(images_clean, axis = 0), axis = (1, 2, 3)))
    diff_ranks = rank_array(diffs)
    overall_rank = diff_ranks + cloud_ranks
    reference_idx = np.argmin(overall_rank)
    ri = arr[reference_idx]
    print(reference_idx)
    
    nir_means = np.mean(arr[:, :, :, 4], axis = (0))
    
    shadows = np.zeros((arr.shape[0], 96, 96))    
    # Candra et al. 2020
    
    for time in range(arr.shape[0]):
        for x in range(arr.shape[1]):
            for y in range(arr.shape[2]):
                ti_slice = arr[time, x, y]
                ri_slice = ri[x, y]
                deltab2 = ti_slice[1] - ri_slice[1]
                deltab3 = ti_slice[2] - ri_slice[2]
                deltab4 = ti_slice[3] - ri_slice[3]
                deltab8a = ti_slice[6] - ri_slice[6]
                deltab11 = ti_slice[8] - ri_slice[8]

                if deltab2 < 0.1:
                    if deltab3 < 0.08:
                        if deltab4 < 0.08:
                            if deltab8a < -0.04:
                                if deltab11 < -0.04:
                                    if ti_slice[1] < 0.0950:
                                        shadows[time, x, y] = 1.
    print(np.sum(shadows, axis = (1, 2)))
                                        
    # Additional 3 time-window NIR thresholding
    
    for time in range(shadows.shape[0]):
        for x in range(shadows.shape[1]):
            for y in range(shadows.shape[2]):
                if shadows[time, x, y] == 1:
                    time_adjusted = np.max([time, 1])
                    time_adjusted = np.min([time_adjusted, shadows.shape[0] - 2])
                    # check before and after
                    if shadows[time_adjusted + 1, x, y] == 1 or shadows[time_adjusted-1, x, y] == 1:
                        nir_mean = nir_means[x, y]
                        nir_before = arr[time_adjusted - 1, x, y, 4] - nir_mean
                        nir_now = arr[time, x, y, 4] - nir_mean
                        nir_next = arr[time_adjusted + 1, x, y, 4] - nir_mean
                        if nir_now > nir_mean * 0.75:
                            shadows[time, x, y] = 0
                        if nir_next or nir_before <= nir_now:
                            shadows[time, x, y] = 0
                         
    print(np.sum(shadows, axis = (1, 2)))
    
    # Remove shadows if cannot coreference a cloud
    shadow_large = np.reshape(shadows, (shadows.shape[0], 96//8, 8, 96//8, 8))
    shadow_large = np.sum(shadow_large, axis = (2, 4))
    
    cloud_large = np.copy(c_probs)
    cloud_large[np.where(c_probs > 0.33)] = 1.
    cloud_large[np.where(c_probs < 0.33)] = 0.
    cloud_large = np.reshape(cloud_large, (shadows.shape[0], 96//8, 8, 96//8, 8))
    cloud_large = np.sum(cloud_large, axis = (2, 4))
    for time in range(shadow_large.shape[0]):
        for x in range(shadow_large.shape[1]):
            x_low = np.max([x - 8, 0])
            x_high = np.min([x + 8, shadow_large.shape[1] - 1])
            for y in range(shadow_large.shape[2]):
                y_low = np.max([y - 8, 0])
                y_high = np.min([y + 8, shadow_large.shape[1] - 1])
                if shadow_large[time, x, y] < 8:
                    shadow_large[time, x, y] = 0.
                if shadow_large[time, x, y] >= 8:
                    shadow_large[time, x, y] = 1.
                c_prob_window = cloud_large[time, x_low:x_high, y_low:y_high]
                if np.max(c_prob_window) < 24:
                    shadow_large[time, x, y] = 0.
                    
    
    shadow_large = resize(shadow_large, (shadow_large.shape[0], 96, 96), order = 0)
    shadows *= shadow_large
    print(np.sum(shadows, axis = (1, 2)))
    
    # Go through and aggregate the shadow map to an 80m grid, and extend it one grid size around
    # any positive ID
    
    shadows_original = np.copy(shadows)
    shadows = np.reshape(shadows, (shadows.shape[0], 96//8, 8, 96//8, 8))
    shadows = np.sum(shadows, axis = (2, 4))
    shadows[np.where(shadows < 16)] = 0.
    shadows[np.where(shadows >= 16)] = 1.
    shadows = resize(shadows, (shadows.shape[0], 96, 96), order = 0)
    shadows = np.reshape(shadows, (shadows.shape[0], 96//4, 4, 96//4, 4))
    shadows = np.max(shadows, (2, 4))
    
    shadows_new = np.zeros_like(shadows)
    for time in range(shadows.shape[0]):
        for x in range(shadows.shape[1]):
            for y in range(shadows.shape[2]):
                if shadows[time, x, y] == 1:
                    min_x = np.max([x - 1, 0])
                    max_x = np.min([x + 2, shadows.shape[1] - 1])
                    min_y = np.max([y - 1, 0])
                    max_y = np.min([y + 2, shadows.shape[1] - 1])
                    for x_idx in range(min_x, max_x):
                        for y_idx in range(min_y, max_y):
                            shadows_new[time, x_idx, y_idx] = 1.
    shadows_new = resize(shadows_new, (shadows.shape[0], 96, 96), order = 0)
    print("The shadow probability is: {}".format(100*np.sum(shadows_new)/(96*96*shadows_new.shape[0])))
    print(np.sum(shadows_new, axis = (1, 2)))
    return shadows_new, shadows_original

def identify_clouds(bbox, epsg = EPSG, time = time):
    #try:
    box = BBox(bbox, crs = epsg)
    cloud_request = WmsRequest(
        layer='CLOUD_DETECTION',
        bbox=box,
        time=time,
        width=96,
        height=96,
        image_format = MimeType.TIFF_d32f,
        maxcc=0.75,
        instance_id=API_KEY,
        custom_url_params = {constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
        time_difference=datetime.timedelta(hours=48),
    )

    cloud_img = cloud_request.get_data()
    print(np.array(cloud_img).shape)
    cloud_probs = cloud_detector.get_cloud_probability_maps(np.array(cloud_img))
    shadows, shadow_original = mcm_shadow_mask(np.array(cloud_img), cloud_probs)
    shadows = shadows[:, 24:-24, 24:-24]
    return cloud_probs[:, 24:-24, 24:-24], shadows, shadow_original, cloud_probs
    #except Exception as e:
    #    logging.fatal(e, exc_info=True)
    
    
def download_dem(val, df, epsg = EPSG, ):
    location = calc_bbox(val, df = df)
    bbox = bounding_box(location[0], expansion = (IMSIZE+2)*10)
    box = BBox(bbox, crs = epsg)
    dem_request = WmsRequest(data_source=DataSource.DEM,
                         layer='DEM',
                         bbox=box,
                         width=IMSIZE+2,
                         height=IMSIZE+2,
                         instance_id=API_KEY,
                         image_format=MimeType.TIFF_d32f,
                         custom_url_params={CustomUrlParam.SHOWLOGO: False})
    dem_image = dem_request.get_data()[0]
    dem_image = calcSlope(dem_image.reshape((1, IMSIZE+2, IMSIZE+2)),
                  np.full((IMSIZE+2, IMSIZE+2), 10), np.full((IMSIZE+2, IMSIZE+2), 10), zScale = 1, minSlope = 0.02)
    dem_image = dem_image.reshape((IMSIZE+2, IMSIZE+2, 1))
    dem_image = dem_image[1:IMSIZE+1, 1:IMSIZE+1, :]
    return dem_image

def check_zenith(bbox, epsg = EPSG, time = time):
    try:
        box = BBox(bbox, crs = epsg)
        zenith = WmsRequest(
            layer='ZENITH',
            bbox=box,
            time=time,
            width=IMSIZE,
            height=IMSIZE,
            image_format = MimeType.TIFF_d32f,
            maxcc=0.75,
            instance_id=API_KEY,
            custom_url_params = {constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
            time_difference=datetime.timedelta(hours=48),
        )
        
        zenith = zenith.get_data()
        return zenith
    except Exception as e:
        logging.fatal(e, exc_info=True)
        
def download_layer(bbox, epsg = EPSG, time = time):
    try:
        box = BBox(bbox, crs = epsg)
        image_request = WcsRequest(
                layer='L2A20',
                bbox=box,
                time=time,
                image_format = MimeType.TIFF_d32f,
                maxcc=0.75,
                resx='10m', resy='10m',
                instance_id=API_KEY,
                custom_url_params = {constants.CustomUrlParam.DOWNSAMPLING: 'NEAREST',
                                    constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
                time_difference=datetime.timedelta(hours=48),
            )
        img_bands = image_request.get_data()
        img_20 = np.stack(img_bands)
        img_20 = resize(img_20, (img_20.shape[0], IMSIZE, IMSIZE, img_20.shape[-1]), order = 0)
        
        image_request = WcsRequest(
                layer='L2A10',
                bbox=box,
                time=time,
                image_format = MimeType.TIFF_d32f,
                maxcc=0.75,
                resx='10m', resy='10m',
                instance_id=API_KEY,
                custom_url_params = {constants.CustomUrlParam.DOWNSAMPLING: 'BICUBIC',
                                    constants.CustomUrlParam.UPSAMPLING: 'BICUBIC'},
                time_difference=datetime.timedelta(hours=48),
        )
        
        img_bands = image_request.get_data()
        img_10 = np.stack(img_bands)
        print("Original size: {}".format(img_10.shape))
        img_10 = resize(img_10, (img_10.shape[0], IMSIZE, IMSIZE, img_10.shape[-1]), order = 0)
        img = np.concatenate([img_10, img_20], axis = -1)
        return img, image_request

    except Exception as e:
        logging.fatal(e, exc_info=True)

# Cloud and shadow removal

In [7]:
def remove_cloud_and_shadows(tiles, probs, shadows, image_dates, wsize = 5):
    c_probs = np.copy(probs)
    c_probs = c_probs - np.min(c_probs, axis = 0)
    c_probs[np.where(c_probs > 0.33)] = 1.
    c_probs[np.where(c_probs < 0.33)] = 0.
    c_probs = np.reshape(c_probs, [c_probs.shape[0], int(IMSIZE/8), 8, int(IMSIZE/8), 8])
    c_probs = np.sum(c_probs, (2, 4))
    c_probs = resize(c_probs, (c_probs.shape[0], IMSIZE, IMSIZE), 0)
    c_probs[np.where(c_probs < 12)] = 0.
    c_probs[np.where(c_probs >= 12)] = 1.
    c_probs += shadows
    c_probs[np.where(c_probs >= 1.)] = 1.
    n_interp = 0
    for cval in range(0, IMSIZE - 4, 1):
        for rval in range(0, IMSIZE - 4, 1):
            subs = c_probs[:, cval:cval + wsize, rval:rval+wsize]
            satisfactory = [x for x in range(c_probs.shape[0]) if np.sum(subs[x, :, :]) < 10]
            satisfactory = np.array(satisfactory)
            for date in range(0, tiles.shape[0]):
                if np.sum(subs[date, :, :]) > 12:
                    n_interp += 1
                    before, after = calculate_proximal_steps_index(date, satisfactory)
                    before = date + before
                    after = date + after
                    bef = tiles[before, cval:cval+wsize, rval:rval+wsize, : ]
                    aft = tiles[after, cval:cval+wsize, rval:rval+wsize, : ]
                    before = image_dates[before]
                    after = image_dates[after]
                    before_diff = abs(image_dates[date] - before)
                    after_diff = abs(image_dates[date] - after)
                    bef_wt = 1 - before_diff / (before_diff + after_diff)
                    aft_wt = 1 - bef_wt
                    candidate = bef_wt*bef + aft_wt*aft
                    candidate = candidate*c_arr + tiles[date, cval:cval+wsize, rval:rval+wsize, : ]*o_arr
                    tiles[date, cval:cval+wsize, rval:rval+wsize, : ] = candidate  
    print("Interpolated {} px".format(n_interp))
    return tiles

def remove_missed_clouds(img):
    iqr = np.percentile(img[:, :, :, 3].flatten(), 75) - np.percentile(img[:, :, :, 3].flatten(), 25)
    thresh_t = np.percentile(img[:, :, :, 3].flatten(), 75) + iqr*2
    thresh_b = np.percentile(img[:, :, :, 3].flatten(), 25) - iqr*2
    diffs_fw = np.diff(img, 1, axis = 0)
    diffs_fw = np.mean(diffs_fw, axis = (1, 2, 3))
    diffs_fw = np.array([0] + list(diffs_fw))
    diffs_bw = np.diff(np.flip(img, 0), 1, axis = 0)
    diffs_bw = np.flip(np.mean(diffs_bw, axis = (1, 2, 3)))
    diffs_bw = np.array(list(diffs_bw) + [0])
    diffs = abs(diffs_fw - diffs_bw) * 100 # 3, -3 -> 6, -3, 3 -> 6, -3, -3
    #diffs = [int(x) for x in diffs]
    outlier_percs = []
    for step in range(img.shape[0]):
        bottom = len(np.argwhere(img[step, :, :, 3].flatten() > thresh_t))
        top = len(np.argwhere(img[step, :, :, 3].flatten() < thresh_b))
        p = 100* ((bottom + top) / (IMSIZE*IMSIZE))
        outlier_percs.append(p)
    to_remove = np.argwhere(np.array(outlier_percs) > 15)
    print([int(x) for x in outlier_percs])
    return to_remove

# Data interpolation

In [8]:
def calculate_and_save_best_images(img_bands, image_dates):
    # This function interpolates data to 5 day windows linearly

    biweekly_dates = [day for day in range(0, 360, 5)] # ideal imagery dates are every 15 days
    
    # Clouds have been removed at this step, so all steps are satisfactory
    satisfactory_ids = [x for x in range(0, img_bands.shape[0])]
    satisfactory_dates = [value for idx, value in enumerate(image_dates) if idx in satisfactory_ids]
    
    
    selected_images = {}
    for i in biweekly_dates:
        distances = [abs(date - i) for date in satisfactory_dates]
        closest = np.min(distances)
        closest_id = np.argmin(distances)
        # If there is imagery within 5 days, select it
        if closest < 8:
            date = satisfactory_dates[closest_id]
            image_idx = int(np.argwhere(np.array(image_dates) == date)[0])
            selected_images[i] = {'image_date': [date], 'image_ratio': [1], 'image_idx': [image_idx]}
        # If there is not imagery within 7 days, look for the closest above and below imagery
        else:
            distances = np.array([(date - i) for date in satisfactory_dates])
            # Number of days above and below the selected date of the nearest clean imagery
            above = distances[np.where(distances < 0, distances, -np.inf).argmax()]
            below = distances[np.where(distances > 0, distances, np.inf).argmin()]
            if abs(above) > 240: # If date is the last date, occassionally argmax would set above to - number
                above = below
            if abs(below) > 240:
                below = above
            if above != below:
                below_ratio = above / (above - below)
                above_ratio = 1 - below_ratio
            else:
                above_ratio = below_ratio = 0.5
                
            # Extract the image date and imagery index for the above and below values
            above_date = i + above
            above_image_idx = int(np.argwhere(np.array(image_dates) == above_date)[0])
            
            below_date = i + below
            below_image_idx = int(np.argwhere(np.array(image_dates) == below_date)[0])
            
            selected_images[i] = {'image_date': [above_date, below_date], 'image_ratio': [above_ratio, below_ratio],
                                 'image_idx': [above_image_idx, below_image_idx]}
                               
    max_distance = 0
    
    for i in selected_images.keys():
        #print(i, selected_images[i])
        if len(selected_images[i]['image_date']) == 2:
            dist = selected_images[i]['image_date'][1] - selected_images[i]['image_date'][0]
            if dist > max_distance:
                max_distance = dist
    
    print("Maximum time distance: {}".format(max_distance))
        
    # Compute the weighted average of the selected imagery for each time step
    keep_steps = []
    use_median = False
    for i in selected_images.keys():
        step1_additional = None
        step2_additional = None
        info = selected_images[i]
        if len(info['image_idx']) == 1:
            step = img_bands[info['image_idx'][0]]
        if len(info['image_idx']) == 2:
            step1 = img_bands[info['image_idx'][0]] # * info['image_ratio'][0]
            step2 = img_bands[info['image_idx'][1]]
            #if info['image_idx'][0] == 0:
            #    step1_additional = img_bands[-1]
            #    print("Using last step")
            #if info['image_idx'][1] == (img_bands.shape[0] - 1):
            #    step2_additional = img_bands[0]
            ##    print("Using first step")
            #if step1_additional is None and step2_additional is None:
            step = step1 * 0.5 + step2 * 0.5
            #if step1_additional is not None:
            #    print("Echo")
            #    step = (step1 + step2 + step1_additional) * (1/3)
            #if step2_additional is not None:
             #   print("Echo")
            #    step = (step1 + step2 + step2_additional) * (1/3)
        keep_steps.append(step)
    '''
    for i in selected_images.keys():
        info = selected_images[i]
        if len(info['image_idx']) == 1:
            step = img_bands[info['image_idx'][0]]
            use_median = False
        if len(info['image_idx']) == 2:
            difference = np.max([abs(info['image_date'][0] - int(i)),
                                 abs(info['image_date'][1] - int(i))]) 
            step1 = img_bands[info['image_idx'][0]] # * info['image_ratio'][0]
            step2_idx = info['image_idx'][0] - 1
            if step2_idx < 0:
                step2_idx = (img_bands.shape[0] - 1)
            step2 = img_bands[step2_idx]
            step3 = img_bands[info['image_idx'][1]]
            step4_idx = info['image_idx'][1] + 1
            if step4_idx > (img_bands.shape[0] - 1):
                step4_idx = 0
            step4 = img_bands[step4_idx]
            #step2 = img_bands[info['image_idx'][1]] * 0.5 # info['image_ratio'][1]
            if difference > 100 or use_median == True:
                print("Median, {}".format(difference))
                use_median = True
                stacked = np.stack([step1, step2, step3, step4])
                step = np.median(stacked, axis = 0)
            else:
                use_median = False
                step = step1 * 0.5 + step3 * 0.5
        '''
        #keep_steps.append(step)
        
    keep_steps = np.stack(keep_steps)
    return keep_steps, max_distance

# Download function

In [9]:
days_per_month = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30]
starting_days = np.cumsum(days_per_month)
print(starting_days)

[  0  31  59  90 120 151 181 212 243 273 304 334]


In [13]:
super_resolve = True
year = 2019

DATA_LOCATION = '../data/ghana-test.csv'
OUTPUT_FOLDER = '../data/test-smooth-200/'


def download_plots(df, output_folder = OUTPUT_FOLDER):
    plot_ids = [x for x in range(0, len(df))][2000:2700]
    existing = [int(x[:-4]) for x in os.listdir(output_folder) if ".DS" not in x]
    to_download = [x for x in plot_ids if x not in existing]
    print("STARTING DOWNLOAD OF {} plots to {}".format(len(to_download), output_folder))
    errors = []
    for i, val in enumerate(to_download):
        print(i, val)
        print("Downloading {}/{}, {}".format(i+1, len(to_download), val))
        location = calc_bbox(val, df = df)
        location = bounding_box(location[0], expansion = IMSIZE*10)
        location_clouds = bounding_box(location[0], expansion = 96*10)
        try:
            # Identify cloud steps, download DEM, and download L2A series
            probs, shadows, shadow_original, prob_original = identify_clouds(location_clouds)
            shadow_sums = np.sum(shadows, axis = (1, 2))
            shadow_steps = np.argwhere(shadow_sums > (48*48/3))
            dem = download_dem(val, df = df)
            img, image_request = download_layer(location)

            # Subset zenith < 70
            zenith = check_zenith(location)
            zenith = np.mean(np.stack(zenith), axis = (1, 2))
            zenith_outliers = np.argwhere(zenith > 70)
            if len(zenith_outliers) > 0:
                print("Zenith outlier: {}".format(zenith_outliers))

            # Calculate imagery dates
            image_dates = []
            for date in image_request.get_dates():
                if date.year == year - 1:
                    image_dates.append(-365 + starting_days[(date.month-1)] + date.day)
                if date.year == year:
                    image_dates.append(starting_days[(date.month-1)] + date.day)
                if date.year == year + 1:
                    image_dates.append(365 + starting_days[(date.month-1)]+date.day)
            image_dates = np.array(image_dates)
            print(image_dates)

            # Remove imagery where >4% is clouds, and where there is null data
            args = np.array([len(np.argwhere(probs[x].flatten() > 0.3)) for x in range(probs.shape[0])])
            dirty_steps = np.argwhere(args > (IMSIZE)*(IMSIZE) / 5)
            missing_images = [np.argwhere(img[x, :, : :].flatten() == 0.0) for x in range(img.shape[0])]
            missing_images = np.array([len(x) for x in missing_images])
            missing_images_p = [np.argwhere(img[x, :, : :].flatten() >= 1) for x in range(img.shape[0])]
            missing_images_p = np.array([len(x) for x in missing_images_p])
            missing_images += missing_images_p
            missing_images = list(np.argwhere(missing_images >= 25))
            to_remove = np.unique(np.array(list(dirty_steps) + list(missing_images) + list(zenith_outliers) + list(shadow_steps)))

            # Remove null steps
            print("There are {}/{} dirty steps: {} cloud, {} missing, {} zenith {} shadow".format(len(to_remove),
                                                                                        len(img), len(dirty_steps),
                                                                                        len(missing_images),
                                                                                        len(zenith_outliers),
                                                                                       len(shadow_steps)))

            img = np.delete(img, to_remove, 0)
            probs = np.delete(probs, to_remove, 0)
            image_dates = np.delete(image_dates, to_remove)
            shadows = np.delete(shadows, to_remove, 0)

            to_remove = remove_missed_clouds(img)
            img = np.delete(img, to_remove, 0)
            probs = np.delete(probs, to_remove, 0)
            image_dates = np.delete(image_dates, to_remove)
            shadows = np.delete(shadows, to_remove, 0)
            print("Removing {} steps based on ratio".format(len(to_remove)))


            # Concatenate DEM
            dem = np.tile(dem.reshape((1, IMSIZE, IMSIZE, 1)), (img.shape[0], 1, 1, 1))
            tiles = np.concatenate([img, dem], axis = -1)
            tiles[:, :, :, -1] /= 90

            x = remove_cloud_and_shadows(tiles, probs, shadows, image_dates)
            if super_resolve:
                x = x[:, 8:40, 8:40, :]
                print("Before super: {}".format(x.shape))

                d10 = x[:, :, :, 0:4]
                d20 = x[:, :, :, 4:10]

                d10 = np.swapaxes(d10, 1, -1)
                d10 = np.swapaxes(d10, 2, 3)
                d20 = np.swapaxes(d20, 1, -1)
                d20 = np.swapaxes(d20, 2, 3)
                superresolved = DSen2(d10, d20)
                superresolved = np.swapaxes(superresolved, 1, -1)
                superresolved = np.swapaxes(superresolved, 1, 2)
                print(superresolved.shape)
                print(x.shape)

                # returns band IDXs 3, 4, 5, 7, 8, 9
                x[:, :, :, 4:10] = superresolved
                x = x[:, 8:24, 8:24, :]
                print("After super shape: {}".format(x.shape))
            else:
                bottom = int(IMSIZE/2 - 8)
                top = int(IMSIZE/2 + 8)
                x = x[:, bottom:top, bottom:top, :]

            # Calculate indices
            tiles, amin = evi(x, True)
            # Where evi is OOB, remove (likely cloud cover missed)
            #if len(amin) > 0:
            #    satisfactory = [x for x in range(tiles.shape[0]) if x not in amin]
            ##    for i in amin:
            #        before, after = calculate_proximal_steps_index(i, satisfactory)
            #        print("Interpolating {} with {} and {}".format(i, before, after))
            #        bef = tiles[before, :, :, :]
            #        aft = tiles[after, :, :, :]
            #        tiles[i, :, :, :] = (bef + aft) / 2

            tiles = bi(tiles, True)
            tiles = msavi2(tiles, True)
            x = si(tiles, True)

            print("Clean: {}".format(x.shape))
            
            missing_pixels = 0
            for band in range(0, 15):
                for time in range(0, x.shape[0]):
                    x_i = x[time, :, :, band]
                    missing_pixels += len(np.argwhere(np.isnan(x_i)))
                    x_i[np.argwhere(np.isnan(x_i))] = np.mean(x_i)
                    x[time, :, :, band] = x_i
            print("There are {} missing pixels".format(missing_pixels))

            # Interpolate linearly to 5 day frequency
            tiles, max_distance = calculate_and_save_best_images(x, image_dates) # 22, 16, 16, 10

            # Smooth linear interpolation
            for row in range(0, 16):
                for column in range(0, 16):
                    for band in [x for x in range(0, 15) if x != 10]:
                        sm = smooth(tiles[:, row, column, band], 800, d = 2)
                        tiles[:, row, column, band] = sm

            # Retain only iamgery every 15 days
            biweekly_dates = np.array([day for day in range(0, 360, 5)])
            to_remove = np.argwhere(biweekly_dates % 15 != 0)
            tiles = np.delete(tiles, to_remove, 0)
            print(tiles.shape)

            if max_distance <= 240:
                np.save(output_folder + str(val), tiles)
                print("\n")
            else:
                print("Skipping {} because there is a {} distance".format(val, max_distance))
                print("\n")

        except Exception as e:
            print(e)
            logging.fatal(e, exc_info=True)
            errors.append(img)
            #continue

In [14]:
x = np.load("../data/train-super/136089019.npy")
sns.heatmap(x[12, :, :, 3])

FileNotFoundError: [Errno 2] No such file or directory: '../data/train-super/136089019.npy'

In [ ]:
download_plots(df, "../../data/fra-s2-stratified/")

STARTING DOWNLOAD OF 700 plots to ../../data/fra-s2-stratified/
0 2000
2000
[-235. -235.]
[245. 245.]
[-235. -235.]
[245. 245.]
(72, 96, 96, 10)
46
[   0.    0.    0.    0.    0.    0.    0.   36.    0.    0.    0.    0.
    0.    0. 4753. 7676.    0.    0.    0.    0.    0.    0. 1643.    0.
  612.    0.  524.    0. 1992.    0.    0.    0.   40.    0.    0.    0.
    0.    0.    0.  174.    0.    0. 5697.    0.    0.    0.    0.  104.
   26.    0.    0.    0.    0.    0. 2682.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0. 4760.    0.]
[   0.    0.    0.    0.    0.    0.    0.   36.    0.    0.    0.    0.
    0.    0.   32. 7676.    0.    0.    0.    0.    0.    0. 1643.    0.
  612.    0.  524.    0. 1992.    0.    0.    0.   40.    0.    0.    0.
    0.    0.    0.  174.    0.    0. 5697.    0.    0.    0.    0.  104.
   26.    0.    0.    0.    0.    0. 2682.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.

Original size: (52, 48, 48, 4)
Zenith outlier: [[ 0]
 [ 1]
 [ 2]
 [43]
 [44]
 [45]
 [46]
 [47]
 [48]
 [49]
 [50]
 [51]]
[-14  -2   6  61  66  73  83  88  91  98 101 106 111 116 121 128 131 138
 141 146 151 156 161 166 171 176 183 188 193 196 206 211 216 221 233 236
 243 246 253 256 266 298 301 311 326 331 336 346 353 356 361 373]
There are 31/52 dirty steps: 27 cloud, 10 missing, 12 zenith 1 shadow
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Removing 0 steps based on ratio
Interpolated 603 px
Before super: (21, 32, 32, 11)
Predicting using file: ../../src/dsen2/models/s2_032_lr_1e-04.hdf5
21/21 [==============================] - 1s 30ms/step
(21, 32, 32, 6)
(21, 32, 32, 11)
After super shape: (21, 16, 16, 11)
There are: 0 out of bounds EVI
Clean: (21, 16, 16, 15)
There are 0 missing pixels
Maximum time distance: 42
(24, 16, 16, 15)


4 2004
2004
[-235. -235.]
[245. 245.]
[-235. -235.]
[245. 245.]
(66, 96, 96, 10)
23
[   0.   34.    0.    0.    0.    0.    0.    0. 7

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
The shadow probability is: 0.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
2007
[-235. -235.]
[245. 245.]
Original size: (37, 49, 47, 4)
[ -9  11  46  61  71  91 101 121 126 131 151 156 166 181 186 196 201 206
 211 216 221 226 231 236 241 246 256 261 266 276 281 286 296 306 311 326
 361]
There are 17/37 dirty steps: 12 cloud, 6 missing, 0 zenith 0 shadow
[4, 1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 19]
Removing 1 steps based on ratio
Interpolated 1031 px
Before super: (19, 32, 32, 11)
Predicting using file: ../../src/dsen2/models/s2_032_lr_1e-04.hdf5
19/19 [==============================] - 1s 36ms/step
(19, 32, 32, 6)
(19, 32, 32, 11)
After super shape: (19, 16, 16, 11)
There are: 0 out of bounds EVI
Clean: (19, 16, 16, 15)
There are 0 missing pixels
Maximum time distance: 65
(24, 16, 16, 15

[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 259.   0.   0.   0.
   0.   0.   0.]
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 213.   0.   0.   0.
   0.   0.   0.]
The shadow probability is: 0.08561643835616438
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  

Interpolated 1263 px
Before super: (46, 32, 32, 11)
Predicting using file: ../../src/dsen2/models/s2_032_lr_1e-04.hdf5
46/46 [==============================] - 1s 30ms/step
(46, 32, 32, 6)
(46, 32, 32, 11)
After super shape: (46, 16, 16, 11)
There are: 0 out of bounds EVI
Clean: (46, 16, 16, 15)
There are 0 missing pixels
Maximum time distance: 20
(24, 16, 16, 15)


15 2015
2015
[-235. -235.]
[245. 245.]
[-235. -235.]
[245. 245.]
(72, 96, 96, 10)
40
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 72.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  

...

Interpolated 129 px
Before super: (28, 32, 32, 11)
Predicting using file: ../../src/dsen2/models/s2_032_lr_1e-04.hdf5
28/28 [==============================] - 1s 33ms/step
(28, 32, 32, 6)
(28, 32, 32, 11)
After super shape: (28, 16, 16, 11)
There are: 0 out of bounds EVI
Clean: (28, 16, 16, 15)
There are 0 missing pixels
Maximum time distance: 75
(24, 16, 16, 15)


630 2630
2630
[-235. -235.]
[245. 245.]
[-235. -235.]
[245. 245.]
(47, 96, 96, 10)
26
[   0.    0.    0.    0.    0.    0.    0.    0.    0.  348.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.  108.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0. 2655.    0.    0.    0.    0.    0.]
[   0.    0.    0.    0.    0.    0.    0.    0.    0.  348.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.  108.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0. 2655.    

Original size: (69, 48, 48, 4)
[-16 -11  -6  -1   4   9  14  19  24  29  34  39  44  49  54  59  64  69
  74  79  84  89  94  99 104 109 114 119 124 129 139 154 164 174 179 184
 189 199 204 214 219 229 234 239 244 249 254 259 264 269 274 279 284 289
 294 299 304 309 314 319 329 339 344 354 359 364 369 374 379]
There are 45/69 dirty steps: 45 cloud, 8 missing, 0 zenith 0 shadow
[0, 4, 53, 0, 0, 0, 3, 1, 0, 0, 0, 0, 3, 0, 11, 9, 0, 0, 1, 0, 8, 0, 36, 15]
Removing 3 steps based on ratio
Interpolated 1193 px
Before super: (21, 32, 32, 11)
Predicting using file: ../../src/dsen2/models/s2_032_lr_1e-04.hdf5
21/21 [==============================] - 1s 31ms/step
(21, 32, 32, 6)
(21, 32, 32, 11)
After super shape: (21, 16, 16, 11)
There are: 0 out of bounds EVI
Clean: (21, 16, 16, 15)
There are 0 missing pixels
Maximum time distance: 55
(24, 16, 16, 15)


635 2635
2635
[-235. -235.]
[245. 245.]
[-235. -235.]
[245. 245.]
(70, 96, 96, 10)
66
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

Original size: (61, 49, 46, 4)
Zenith outlier: [[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]
 [11]
 [54]
 [55]
 [56]
 [57]
 [58]
 [59]
 [60]]
[ -9  -4  -1   4   9  16  19  24  31  34  41  44  49  66  69  74  79  91
  96  99 106 109 111 114 131 134 139 144 149 161 166 169 181 186 189 196
 199 204 209 216 224 231 239 244 251 254 259 264 271 276 279 284 291 299
 309 314 324 341 344 349 361]
There are 40/61 dirty steps: 28 cloud, 15 missing, 19 zenith 1 shadow
[98, 93, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0]
Removing 2 steps based on ratio
Interpolated 700 px
Before super: (19, 32, 32, 11)
Predicting using file: ../../src/dsen2/models/s2_032_lr_1e-04.hdf5
19/19 [==============================] - 1s 32ms/step
(19, 32, 32, 6)
(19, 32, 32, 11)
After super shape: (19, 16, 16, 11)
There are: 0 out of bounds EVI
Clean: (19, 16, 16, 15)
There are 0 missing pixels
Maximum time distance: 35
(24, 16, 16, 15)


639 2639
2639
[-235. -235.]
[245. 245.]
[-235. -235.]

Original size: (72, 48, 48, 4)
[-13  -8  -3   2   7  12  17  22  27  32  37  42  47  52  57  62  67  77
  82  87  97 102 107 117 122 127 132 137 157 162 167 172 177 187 192 197
 202 207 212 217 222 227 232 237 242 247 252 257 262 267 272 277 282 287
 292 297 302 307 312 317 322 327 332 337 342 347 352 357 362 367 372 377]
There are 46/72 dirty steps: 46 cloud, 8 missing, 0 zenith 0 shadow
[0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 5, 21, 0, 0, 3, 0, 1, 0, 0, 25, 35, 0, 13, 5, 5, 0]
Removing 3 steps based on ratio
Interpolated 799 px
Before super: (23, 32, 32, 11)
Predicting using file: ../../src/dsen2/models/s2_032_lr_1e-04.hdf5
23/23 [==============================] - 1s 29ms/step
(23, 32, 32, 6)
(23, 32, 32, 11)
After super shape: (23, 16, 16, 11)
There are: 0 out of bounds EVI
Clean: (23, 16, 16, 15)
There are 0 missing pixels
Maximum time distance: 65
(24, 16, 16, 15)


643 2643
2643
[-235. -235.]
[245. 245.]
[-235. -235.]
[245. 245.]
(49, 96, 96, 10)
34
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

(48, 96, 96, 10)
22
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 95.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 95.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 91.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
The shadow probability is: 0.09403935185185185
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 416.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.]
2646
[-235. -235.]
[245. 245.]
